(always be aware of your imports and <b><u><i>preserve namespaces</i></u></b>!!!)

In [1]:
import os
import sys
import time
import numpy as np
import scipy.ndimage as nd
import matplotlib.pyplot as plt

%matplotlib tk

plt.ion()
plt.rcParams["image.cmap"] = "gist_gray"

---

## Multiple frames (videos)

Unlike single still images, multiple images of the same scene over time let you measure **time dependent** features (e.g., motion, color changes, etc.).  We will deal with already extracted frames from a video here, but an example about how that extraction can be done in python with OpenCV is <a href="http://tobilehman.com/blog/2013/01/20/extract-array-of-frames-from-mp4-using-python-opencv-bindings/">here</a> (old link, but the code still works). 

### DOT camera analysis

The Department of Transportation has hundreds of traffic cameras located around the city.  Using this script:

    import os
    import time

    nframe   = 100
    cmd_wget = "wget http://207.251.86.238/cctv391.jpg"
    cmd_mv   = "mv cctv391.jpg images/dot/cctv391_{0:03}.jpg"

    for ii in range(nframe):
        os.system(cmd_wget)
        os.system(cmd_mv.format(ii))
        time.sleep(1)

I grabbed 100 frames from camera overlooking the Manhattan bridge (and Lower East Side):

In [2]:
path1 = "images"
path2 = "dot"
dpath = os.path.join(path1, path2)
flist = [os.path.join(dpath, i) for i in 
         sorted(os.listdir(os.path.join(dpath)))]

for fname in flist:
    print(fname)

images/dot/cctv391_000.jpg
images/dot/cctv391_001.jpg
images/dot/cctv391_002.jpg
images/dot/cctv391_003.jpg
images/dot/cctv391_004.jpg
images/dot/cctv391_005.jpg
images/dot/cctv391_006.jpg
images/dot/cctv391_007.jpg
images/dot/cctv391_008.jpg
images/dot/cctv391_009.jpg
images/dot/cctv391_010.jpg
images/dot/cctv391_011.jpg
images/dot/cctv391_012.jpg
images/dot/cctv391_013.jpg
images/dot/cctv391_014.jpg
images/dot/cctv391_015.jpg
images/dot/cctv391_016.jpg
images/dot/cctv391_017.jpg
images/dot/cctv391_018.jpg
images/dot/cctv391_019.jpg
images/dot/cctv391_020.jpg
images/dot/cctv391_021.jpg
images/dot/cctv391_022.jpg
images/dot/cctv391_023.jpg
images/dot/cctv391_024.jpg
images/dot/cctv391_025.jpg
images/dot/cctv391_026.jpg
images/dot/cctv391_027.jpg
images/dot/cctv391_028.jpg
images/dot/cctv391_029.jpg
images/dot/cctv391_030.jpg
images/dot/cctv391_031.jpg
images/dot/cctv391_032.jpg
images/dot/cctv391_033.jpg
images/dot/cctv391_034.jpg
images/dot/cctv391_035.jpg
images/dot/cctv391_036.jpg
i

In [3]:
imgs = np.array([nd.imread(i) for i in flist])
nimg, nrow, ncol = imgs.shape[0:3]
xs = 5
ys = 2 * xs * float(nrow) / float(ncol)

plt.close(0)
fig0, ax0 = plt.subplots(2, 1, num=0, figsize=[xs, ys])
fig0.subplots_adjust(0, 0, 1, 1, 0, 0)
[i.axis("off") for i in ax0]
im0a = ax0[0].imshow(imgs[0])
fig0.canvas.draw()

In [5]:
for img in imgs:
    im0a.set_data(img)
    fig0.canvas.draw()
    time.sleep(0.02)

We'd like to **roughly** count the number of cars on this section of the highway.  Let's first look at frame differences:

In [6]:
im0b = ax0[1].imshow(imgs[1].mean(-1) - imgs[0].mean(-1))
fig0.canvas.draw()

In [7]:
for ii in range(1, nimg):
    im0a.set_data(imgs[ii])
    im0b.set_data(imgs[ii].mean(-1) - imgs[ii-1].mean(-1))
    fig0.canvas.draw()
    time.sleep(0.02)

Or taking the absolute value (note: if we were to count off of this we'd be roughly double counting...)

In [8]:
im0b.set_clim(0, 128)

for ii in range(1, nimg):
    im0a.set_data(imgs[ii])
    im0b.set_data(np.abs(imgs[ii].mean(-1) - 
                         imgs[ii-1].mean(-1)))
    fig0.canvas.draw()
    time.sleep(0.02)

Let's do some thresholding:

In [9]:
dimg = np.zeros([nrow, ncol])
im0b.set_clim(0, 1)

for ii in range(1, nimg):
    dimg[:, :] = np.abs(imgs[ii].mean(-1) - 
                        imgs[ii-1].mean(-1))
    im0a.set_data(imgs[ii])
    im0b.set_data(dimg > 40)
    fig0.canvas.draw()
    time.sleep(0.02)

### Background subtraction

But again, we're double counting and adding noise.  Another method is to subtract the "mean image" from each frame:

In [10]:
mimg = imgs.mean(0)

im0a.set_data(mimg.clip(0, 255).astype(np.uint8))
fig0.canvas.draw()

In [11]:
for ii in range(1, nimg):
    im0a.set_data(imgs[ii])
    im0b.set_data(np.abs(1.0 * imgs[ii] - mimg) \
                  .clip(0, 255).astype(np.uint8))
    fig0.canvas.draw()
    time.sleep(0.02)

Let's take the max of this difference in color space:

In [12]:
im0b.set_clim([0, 255])

for ii in range(1, nimg):
    im0a.set_data(imgs[ii])
    im0b.set_data(np.abs(1.0 * imgs[ii] - mimg).max(-1))
    fig0.canvas.draw()
    time.sleep(0.02)

Again, we can do some clipping, but let's take a look at the distribution of brightnesses

In [13]:
fig1, ax1 = plt.subplots(num=1)
ax1.hist(np.log10(np.abs(1.0 * imgs - mimg) \
                  .max(-1).flatten() + 1.0), bins=255)
fig1.canvas.draw()

The logarithm of the difference values for the objects is something like $>1.5 \approx 31$:

In [16]:
thr = 30
im0b.set_clim([0, 1])

for ii in range(1, nimg):
    im0a.set_data(imgs[ii])
    im0b.set_data(np.abs(1.0 * imgs[ii] - mimg).max(-1) > thr)
    fig0.canvas.draw()
    time.sleep(0.02)

In [17]:
bdilation = nd.morphology.binary_dilation
berosion = nd.morphology.binary_erosion

In [18]:
thr = 30
im0b.set_clim([0, 1])
fgr = np.zeros([nrow, ncol], dtype=int)

for ii in range(1, nimg):
    fgr[:, :] = bdilation(berosion(np.abs(1.0 * imgs[ii] - 
                                          mimg).max(-1) > thr), 
                          iterations=2)
    im0a.set_data(imgs[ii])
    im0b.set_data(fgr)
    fig0.canvas.draw()
    time.sleep(0.02)

We can use our trick of a row map to generate a mask,

In [19]:
col_mask = (np.arange(nrow * ncol) \
            .reshape(nrow, ncol) % ncol) < 250

In [20]:
thr = 30
im0b.set_clim([0, 1])
fgr = np.zeros([nrow, ncol], dtype=int)

for ii in range(1, nimg):
    fgr[:, :] = bdilation(berosion(np.abs(1.0 * imgs[ii] - 
                                          mimg).max(-1) > thr), 
                          iterations=2)
    im0a.set_data(imgs[ii])
    im0b.set_data(fgr * col_mask)
    fig0.canvas.draw()
    time.sleep(0.02)

Lastly, we count the number of detected objects:

In [21]:
count = ax0[1].text(0, 0, "# of cars: ", va="top",
                    fontsize=20, color="white")
fig0.canvas.draw()

In [22]:
meas_label = nd.measurements.label

In [23]:
thr = 30
sz_thr = 20 # only count objects greater than a size threshold
im0b.set_clim([0, 1])
fgr = np.zeros([nrow, ncol], dtype=int)

for ii in range(1, nimg):
    fgr[:, :] = bdilation(berosion(np.abs(1.0 * imgs[ii] - 
                                          mimg).max(-1) > thr), 
                          iterations=2)
    labs = meas_label(fgr * col_mask)
    ncar = sum([1 * ((labs[0] == lab).sum() > sz_thr) for lab 
                in range(1, labs[1] + 1)])
    im0a.set_data(imgs[ii])
    im0b.set_data(fgr * col_mask)
    count.set_text("# of cars: {0}".format(ncar))
    fig0.canvas.draw()
    time.sleep(0.02)

---

## Closing thoughts and recap

Below are some of the key concepts we've worked through.  They are broadly applicable, even well beyond the field of image processing and computer vision:

- best practices for filesystem handling and namespaces
- vectorized coding (indexing arrays to eliminate for loops)
- object oriented plotting with matplotlib (for interactive plots)
- thresholding for data subselection and masking
- derivatives and edge detection through array shifts
- filtering (mean, Gaussian, median) in 1D and 2D
- regressions using the normal equation and some linear algebra
- combining 2D data with time series information